In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import scipy.signal as signal
import pandas as pd
import visuals as vs
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from spark_sklearn import GridSearchCV

random_state = 6
np.random.seed(random_state)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [19]:
train_file = 'train.csv'
test_file = 'test.csv'
submission_file = 'sample_submission.csv'
df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)
submission = pd.read_csv(submission_file)

In [15]:
n_feature = (df_train.shape)[1]

X = df_train.drop(['id','target'], axis = 1)
Y = df_train['target']
X_train, Y_train = shuffle(X, Y, random_state = random_state)

# testing data
X = df_train.drop(['id','target'], axis = 1)
Y = df_train['target']
X_test,Y_test = shuffle(X, Y, random_state = random_state)


mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

In [2]:
from pyspark import SparkContext, SparkConf
import os

conf = SparkConf()
sc = SparkContext()

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
rfc = RandomForestClassifier(criterion='entropy', random_state = random_state, class_weight='balanced')
params = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': [1, 2, 3, 4, 5],
    'min_samples_split': [10, 20, 40, 50]
}

grid_clf = GridSearchCV(sc, rfc, param_grid = params, scoring = 'accuracy', cv = cv)

In [8]:
# we have to find out best optimized parameters
grid_clf.fit(X_train, Y_train.astype(int))

NameError: name 'grid_clf' is not defined

In [16]:
# class_weight = {0:1, 1:2}
model = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=random_state, verbose=0, warm_start=False)

model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=6, verbose=0, warm_start=False)

In [26]:
import sys

# Y_pred = model.predict(X_test)
test_proba = model.predict_proba(X_test)
gb_out = submission
print(np.shape(gb_out))
print(np.shape(test_proba))
sys.exit()
gb_out['target'] = test_proba
gb_out['target'] = 1-gb_out['target']
gb_out.to_csv('gb_predictions1.csv', index=False, float_format='%.4f')

(892816, 2)
(595212, 2)


SystemExit: 

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print('The accuracy of the model is {}'.format(accuracy_score(Y_test, Y_pred)))
print('Classification report \n {}'.format(classification_report(Y_test, Y_pred)))
print('Confusion matrix \n {}'.format(confusion_matrix(Y_pred, Y_test)))

In [ ]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
     assert( len(actual) == len(pred) )
     all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
     all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
     totalLosses = all[:,0].sum()
     giniSum = all[:,0].cumsum().sum() / totalLosses
 
     giniSum -= (len(actual) + 1) / 2.
     return giniSum / len(actual)
 
 def gini_normalized(a, p):
     return gini(a, p) / gini(a, a)